In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import dhg

from einops import rearrange, repeat
from einops.layers.torch import Rearrange

In [4]:
class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim) #Normalization on every pixel using mean and S.D
        self.fn = fn                  #Some Layer or Nueral Network
        
    def forward(self, image, **kwargs):
        return self.fn(self.norm(image), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, drop=0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(drop),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(drop)
        )
    def forward(self, img):
        return self.net(img)


class Attention(nn.Module):
    """
    INPUT.shape = OUTPUT.shape = (batch, sequence Length, Feature dim)
    * Multi-Head Attention: Computes multiple attention scores 
        *head: No. of independent attention mechanism applied on each node(GAT)
        *head: No. of independent attention mechanism applied on each patch of an image(Transformet)
        
        *dim_head: No. of dimensions for each head (if dim=512, head=8 each attention head will process 64 features(512/8))
        
    """
    def __init__(self, dim, heads=8, dim_head=64, dropout=0.):
        # head: No. of attention co
        super().__init__()
        inner_dim = dim_head*heads
        project_out = not(heads==1 and dim_head==dim)
        
        self.heads = heads
        self.scale = dim_head ** -0.5
        
        self.attend = nn.Softmax(dim=-1)
        self.to_qkv = nn.Linear(dim, inner_dim*3, bias=False)
        
        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()
        
    def forward(self, x):
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer(nn.Module):
    """
    * INPUT.shape = OUTPUT.shape = (Batch, Seqence Length, Feature Dim)
    * depth: Depth of the transformer
    * ModuleList: Very identical to list but recognized as a part of model
    """
    def __init__(self, dim, depth, heads, dim_heads, mlp_dim, dropout=0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads=heads, dim_head=dim_heads, dropout=dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout))
            ]))
    def forward(self, x):
        for att, ff in self.layers:
            x = att(x) + x # Adds for residual connection
            x = ff(x) + x # Adds for residual connection
        return x

def pair(t):
    return t if isinstance(t, tuple) else (t, t)
class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 200, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
            nn.Linear(patch_dim, dim),
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, dim)
        )

        def forward(self, img):
            x = self.to_patch_embedding(img)
            b, n, _ = x.shape

            cls_tokens = repeat(self.cls_token, '() n d -> b n d', b = b)
            x = torch.cat((cls_tokens, x), dim=1)
            x += self.pos_embedding[:, :(n + 1)]
            x = self.dropout(x)

            x = self.transformer(x)

            x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

            x = self.to_latent(x)
            return self.mlp_head(x)